In [0]:
from pyspark.sql import SparkSession, functions as f

#Reading Hospital A departments data
df_hosa= spark.read.parquet("dbfs:/mnt/bronze/hosa/departments")

#Reading Hospital B departments data
df_hosb= spark.read.parquet("dbfs:/mnt/bronze/hosb/departments")

#union two departments dataframes
df_merged= df_hosa.unionByName(df_hosb)

# create the dept_id column and rename deptid to src_dept_id
df_merged= df_merged.withColumn("SRC_Dept_id" , f.col("deptid")) \
                    .withColumn("Dept_id" , f.concat(f.col("deptid"), f.lit("-"), f.col("datasource"))) \
                    .drop("deptid")

df_merged.createOrReplaceTempView("departments")

In [0]:
df_hosb.count()

In [0]:
%sql
CREATE SCHEMA IF NOT EXISTS silver;

CREATE TABLE if not exists silver.departments (
  Dept_id string,
  SRC_Dept_id string,
  Name string,
  datasource string,
  is_quarantined BOOLEAN
)
USING DELTA

In [0]:
%sql
DESCRIBE HISTORY silver.departments ;


In [0]:
%sql

truncate table silver.departments;;

In [0]:
%sql
truncate table silver.departments;;
INSERT INTO silver.departments 
SELECT 
Dept_id , 
SRC_Dept_id, 
Name, 
Datasource, 
    CASE 
        WHEN SRC_Dept_id IS NULL Or Name IS NULL THEN true
        ELSE false
        END as is_quarantined
from departments

In [0]:
%sql

select * from silver.departments 